In [45]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as grb
from gurobipy import GRB

# Input Problem Parameters

## Définissons les variables

In [46]:
from utils.extract_parameters import extract_parameters

################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "large"
# taille = "medium"
# taille = "large"

In [47]:
################################
######## DO NOT CHANGE #########
################################
conge, qualif, d, p, b, besoin = extract_parameters(taille)

In [48]:
print("conge : ",conge)
print("qualif : ",qualif)
print("date livraison : ",d)
print("pénalité : ",p)
print("bénéfice : ",b)
print("besoin : ",besoin)

conge :  [[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]]
qualif :  [[1, 1, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1], [0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1]]
date livraison :  [26, 20, 32, 32, 26, 30, 26, 36, 18, 18, 22, 26, 18, 12, 28, 26, 12, 14, 14, 16, 18, 14, 32, 18, 30]


## Définissons les paramètres

In [49]:
from utils.define_nums import define_nums

nombre_employes, horizon, nombre_qualifs, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

In [50]:
print("nombre_employes : ",nombre_employes)
print("horizon : ",horizon)
print("nombre_qualifs : ",nombre_qualifs)
print("nombre_projets : ",nombre_projets)

nombre_employes :  6
horizon :  36
nombre_qualifs :  10
nombre_projets :  25


In [51]:
## u_j, le projet j est réalisé
u_shape = nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x_shape = [(i,j,k, n) for i in range(nombre_qualifs) for j in range(nombre_projets) for k in range(nombre_employes) for n in range(horizon)]

## t_k_j, l'employé k travaille sur le projet j
t_shape = [(i,j) for i in range(nombre_employes) for j in range(nombre_projets)]

## begin_j, la date de début effective
begin_shape = nombre_projets

## end_j, la date de fin effective = date de livraison du projet j
end_shape = nombre_projets

## delay_j, le nombre de jours de retard par projet j
delay_shape = nombre_projets

## longueur_j, la longueur du projet j
longueur_shape = nombre_projets

## projet_par_employe_k, le nombre de projet par employe
projet_par_employe_shape = nombre_employes

# majorant pour la contrainte de couverture
maj = max([besoin[i][j] for j in range(nombre_projets) for i in range(nombre_qualifs)]) + 1

# si le projet j est travaillé au jour n
job_in_process_shape = [(j, n) for j in range(nombre_projets) for n in range(horizon)]

# Model deployment

## Add the variables

In [52]:
model = grb.Model("Planning")

# L'affectation du planning, la variable clé
x = model.addVars(x_shape, vtype=GRB.BINARY, name="x")

## Les dates de fin des projets
dateFin = model.addVars(end_shape, lb = 0, ub = horizon, name = "dateFin")

## Les dates de début des projets
dateDebut = model.addVars(begin_shape, lb = 0, ub = horizon, name = "dateDebut")

## Le nombre de jours de delai
delay = model.addVars(delay_shape, lb = 0, ub = horizon, name = "delay")

## Les longueurs des projets
longueur = model.addVars(longueur_shape, lb = 0, ub = horizon, name = "longueur")

## Nombre de projet par employé
projet_par_employe = model.addVars(projet_par_employe_shape, lb = 0, ub = nombre_projets, name = "projet_par_employe")

## L'affectation sur un projet d'un employé
affectation = model.addVars(t_shape, vtype = GRB.BINARY, name = "affectation")

## La réalisation ou non d'un projet
u = model.addVars(u_shape, vtype = GRB.BINARY, name="u")

## Verification du projet
job_in_process = model.addVars(job_in_process_shape, vtype = GRB.BINARY, name = "job_in_process")

In [53]:
# Update le modele pour confirmer la création des variables
model.update()


## Add constraints

In [54]:
for i in range(nombre_qualifs):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                ## La qualification du personne
                model.addConstr( x[i,j,k,n] <= qualif[i][k], name="QualificationPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                model.addConstr( x[i,j,k,n] <= (besoin[i][j] >=1), name = "BesoinQualifPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                ## Contraintes des congés
                model.addConstr( x[i,j,k,n] <= conge[k][n], name=f"CongesPersonnel[{i}{j}{k}{n}]")
        
        ## Contrainte d'unicité de réalisation d'un projet
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) == besoin[i][j]*u[j], name=f"UniciteRealisation[{i}{j}]")

        ## Couverture des qualifications
        # model.addConstr( grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) - besoin[i][j] >= maj*(1-u[j]), name=f"CouvertureQualificationsInf[{i}{j}]") 


## Unicité de l'affectation quotidienne
for k in range(nombre_employes):
    for n in range(horizon):
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for j in range(nombre_projets)) for i in range(nombre_qualifs)) <= 1, name=f"UniciteAffectation[{k}{n}]")

        
for j in range(nombre_projets):
    for n in range(horizon):
        model.addConstrs(((job_in_process[j,n] >= x[i,j,k,n]) for i in range(nombre_qualifs) for k in range(nombre_employes)), name=f"ContrainteJobInProcess[{j}{n}]")
    # start date
    model.addConstrs(((dateDebut[j] <= job_in_process[j,n]*n) for n in range(horizon)), name = f"ContrainteDateDebut[{j}]")

    # end date
    model.addConstrs(((job_in_process[j,n]*n <= dateFin[j]) for n in range(horizon)), name=f"FinProjet[{j}]")

    ## longueur
    model.addConstr(longueur[j] == dateFin[j] - dateDebut[j], name=f"LongueurProjet[{j}]")

    ## delay
    model.addConstr(delay[j] >= dateFin[j] - (d[j] - 1), name=f"DelaiProjet_difference[{j}]")    

## projet_par_employe
for k in range(nombre_employes):
    model.addConstr(projet_par_employe[k] == grb.quicksum( affectation[k,j] for j in range(nombre_projets)), name=f"FinProjet[{j}]")
    
    
## Contrainte de l'affectation à un projet selon les jours de travail
for k in range(nombre_employes):
    for j in range(nombre_projets):
        model.addConstr(affectation[k,j] <= grb.quicksum(grb.quicksum(x[i,j,k,n] for i in range(nombre_qualifs)) for n in range(horizon)), name=f"AffectationProjetQuicksum[{k}{j}]")
        
    

## Fonctions objectif

In [55]:
## bénéfice total
benefice_tot = grb.quicksum( (b[j]*u[j] - delay[j]*p[j]) for j in range(nombre_projets) )

# -- Correction -- début --
# Rq : je commente ces lignes pour me focaliser juste sur le premier objectif à savoir le bénéfice total
## Nombre de projet maximum par personne
# projet_max = model.addVar(name="projet_max")
# model.addConstrs(projet_max>=projet_par_employe[k] for k in range(nombre_employes))
# model.update()

## Compacité
# longueur_max = grb.max_(longueur)

# model.ModelSense = GRB.MINIMIZE
# model.setObjectiveN(benefice_tot,0,2)
#model.setObjectiveN(projet_max, 1, 1)
#model.setObjectiveN(longueur_max, 2, 0)
# -- Correction -- fin --

# -- Correction -- début --
# Rq : j'ajoute ces lignes pour me focaliser juste sur le premier objectif à savoir le bénéfice total
model.ModelSense = GRB.MAXIMIZE
model.setObjective(benefice_tot)
model.ModelSense = GRB.MAXIMIZE
# -- Correction -- fin --

# Model optimisation

In [56]:
# Verify model formulation

nom = "results/"+taille+".lp"
model.write(nom)

# Run optimization engine

model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 218472 rows, 55181 columns and 436041 nonzeros
Model fingerprint: 0x3665eabc
Variable types: 106 continuous, 55075 integer (55075 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e+00, 8e+01]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 216898 rows and 50863 columns
Presolve time: 0.18s
Presolved: 1574 rows, 4318 columns, 10996 nonzeros
Variable types: 0 continuous, 4318 integer (4294 binary)

Root relaxation: objective 8.376303e+02, 3162 iterations, 0.06 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [57]:
# # Print the values of all variables
# for v in model.getVars():
#     print(f"{v.VarName} = {v.X}")

# -- Correction -- début --
# Rq : pour voir quels sont les projets faits
# for j in range(nombre_projets):
#     v = model.getVarByName(f"u[{j}]")
#     if int(v.x) == 1:
#         print(f"la tâche {j} est faite")
#     else:
#         print(f"la tâche {j} n'est pas faite")
# -- Correction -- fin --

# -- Correction -- début --
# Rq : pour voir quels sont les edt des employés et le détail de la réalisation de chaque tâche (de façon ordonnée)
schedules = []
tasks_performances = []
for i in range(nombre_qualifs):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                v = model.getVarByName(f"x[{i},{j},{k},{n}]")
                if int(v.x) == 1:
                    schedules.append((k, n, j, i))
                    tasks_performances.append((j, i, k, n))
schedules.sort()
for (k, n, j, i) in schedules:
    print(f"l'employé {k} fait, à la date {n}, la tâche {j} pour la qualif {i}")
tasks_performances.sort()
for (j, i, k, n) in tasks_performances:
    print(f"la tâche {j} pour la qualif {i} est faite par {k} à la date {n}")
# -- Correction -- fin --

# -- Correction -- début --
# Rq : pour voir quelles sont les dates de début et de fin des tâches réalisees 
# (à n'étudier que lorsque longueur_max est mis en objectif, sinon les variables dateDebut et dateFin ne sont pas assez contraintes pour prendre exactement le sens souhaité)
for j in range(nombre_projets):
    v = model.getVarByName(f"u[{j}]")
    v1 = model.getVarByName(f"dateDebut[{j}]")
    v2 = model.getVarByName(f"dateFin[{j}]")
    if int(v.x) == 1:
        print(f"la tâche {j} commence à la date {int(v1.x)} et finit à la date {int(v2.x)}")
# -- Correction -- fin --

l'employé 0 fait, à la date 0, la tâche 10 pour la qualif 0
l'employé 0 fait, à la date 6, la tâche 21 pour la qualif 2
l'employé 0 fait, à la date 7, la tâche 1 pour la qualif 2
l'employé 0 fait, à la date 8, la tâche 1 pour la qualif 2
l'employé 0 fait, à la date 9, la tâche 20 pour la qualif 0
l'employé 0 fait, à la date 10, la tâche 21 pour la qualif 1
l'employé 0 fait, à la date 11, la tâche 21 pour la qualif 2
l'employé 0 fait, à la date 12, la tâche 19 pour la qualif 0
l'employé 0 fait, à la date 13, la tâche 21 pour la qualif 2
l'employé 0 fait, à la date 14, la tâche 19 pour la qualif 0
l'employé 0 fait, à la date 15, la tâche 20 pour la qualif 0
l'employé 0 fait, à la date 16, la tâche 1 pour la qualif 2
l'employé 0 fait, à la date 17, la tâche 20 pour la qualif 0
l'employé 0 fait, à la date 18, la tâche 10 pour la qualif 0
l'employé 0 fait, à la date 19, la tâche 1 pour la qualif 2
l'employé 0 fait, à la date 20, la tâche 10 pour la qualif 1
l'employé 0 fait, à la date 21, l

In [58]:
model.objVal

817.0